In [1]:
# Import the requests library.
import requests
import numpy as np
import pandas as pd
# Import the requests library.
import requests
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
# Dependencies for the wine API
import urllib
import json
# Import the API key.
from config import weather_api_key
from config import Token_NOAA
from config import API_Token
import calendar

In [2]:
# https://github.com/crvaden/NOAA_API_v2
# https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859
# https://cran.r-project.org/web/packages/rnoaa/rnoaa.pdf
# file:///C:/Users/15124/Downloads/GHCND_documentation.pdf
# https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt

In [3]:
# Import the API key.
headers = {
    'accept': 'application/json',
    'authorization': API_Token
    }

# Red Wine in USA API

In [4]:
#import JSON data with the API
req_red_USA = urllib.request.Request('https://api.globalwinescore.com/globalwinescores/latest/?color=red&limit=5000&country=Usa', data=None, headers=headers, origin_req_host=None, unverifiable=False, method=None)
res_red_USA = urllib.request.urlopen(req_red_USA).read().decode()
obj_red_USA = json.loads(res_red_USA)
#read json into DF
red_USA_df = pd.DataFrame.from_dict(obj_red_USA['results'], orient='columns')
# Delete unnecessary columns
red_USA_df.drop([ 'wine_slug', 'appellation_slug',  'date',  'lwin', 'lwin_11'],axis=1, inplace=True)
##'wine_id','wine_type','classification',
##Drop NA values
# red_USA_df = red_USA_df.dropna()
#convert vintage to date type
red_USA_df['vintage']= pd.to_datetime(red_USA_df['vintage']).dt.year
red_USA_df['regions']=red_USA_df['regions'].str.get(0)
# Read the DF to a csv
red_USA_df.to_csv("Red_Wine_USA.csv")
red_USA_df.head(2)

,wine,wine_id,appellation,color,wine_type,regions,country,classification,vintage,is_primeurs,score,confidence_index,journalist_count
0,"Ridge Vineyards, Monte Bello, Santa Cruz Mount...",120786,Santa Cruz Mountains,Red,,California,Usa,None,2013,False,95.83,C+,8
1,"Quintessa, Rutherford",118360,Rutherford,Red,,California,Usa,None,2015,False,95.77,C,3


# White Wine in USA API

In [5]:
#import JSON data with the API
req_white_USA = urllib.request.Request('https://api.globalwinescore.com/globalwinescores/latest/?color=white&limit=5000&country=Usa', data=None, headers=headers, origin_req_host=None, unverifiable=False, method=None)
res_white_USA = urllib.request.urlopen(req_white_USA).read().decode()
obj_white_USA = json.loads(res_white_USA)
#read json into DF
white_USA_df = pd.DataFrame.from_dict(obj_white_USA['results'], orient='columns')
# Delete unnecessary columns
white_USA_df.drop([ 'wine_slug', 'appellation_slug',  'date',  'lwin', 'lwin_11'],axis=1, inplace=True)
##'wine_id','wine_type','classification',
##Drop NA values
# white_USA_df = white_USA_df.dropna()
#convert vintage to date type
white_USA_df['vintage']= pd.to_datetime(white_USA_df['vintage']).dt.year
white_USA_df['regions']=white_USA_df['regions'].str.get(0)
# Read the DF to a csv
white_USA_df.to_csv("White_Wine_USA.csv")
white_USA_df.head(2)

,wine,wine_id,appellation,color,wine_type,regions,country,classification,vintage,is_primeurs,score,confidence_index,journalist_count
0,"Ridge Vineyards, Monte Bello Chardonnay, White...",120785,Santa Cruz Mountains,White,dry,California,Usa,None,2016,False,94.39,B,3
1,"Stags' Leap Winery, Chardonnay, White, Napa Va...",129823,Napa Valley,White,dry,California,Usa,None,2016,False,89.83,A+,3


# Weather Data By Zip Code

In [40]:
bigdf = pd.DataFrame()
date_max_temp= pd.DataFrame()
date_min_temp= pd.DataFrame()
states = { 
    'CA': {
        'zip': '97218',
        'years': ['2010', '2011']
        },
    'Wash': {
        'zip': '99336',
        'years': ['2010', '2011']
        }
    #etc...
}
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+str(year))
        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=TMIN&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
        #Load JSON data
        d = json.loads(r.text)
        #     print(d)
        #e = json.loads(p.text)
        items_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
        items_MIN = [item for item in d['results'] if item['datatype']=='TMIN']

        # #get the date field from all average temperature readings
        list_max_temp = [item['date'] for item in items_MAX]
        list_min_temp = [item['date'] for item in items_MIN]
        
        date_max_temp['date']=[datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in list_max_temp]
        date_min_temp['date']=[datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in list_min_temp]

        df_test['date'] = date_max_temp['date']
        df_test['avgMaxTemp'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMaxTemp'+state]] = item['value']

        df_test['avgMinTemp'+state] = np.nan
        for item in items_MIN:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMinTemp'+state]] = item['value']
        small_df = pd.concat([small_df,df_test])
    if bigdf.empty:
        bigdf = small_df
    else:
        bigdf = pd.merge(bigdf,small_df)
bigdf = bigdf.set_index('date').resample('M').mean()
bigdf.head(2)

working on year 2010
working on year 2011
working on year 2010
working on year 2011


,avgMaxTempCA,avgMinTempCA,avgMaxTempWash,avgMinTempWash
date,,,,
2010-01-31,49.645161,40.451613,44.806452,33.741935
2010-02-28,54.142857,39.000000,50.821429,33.785714


# Add Lat/Long/Elevation

In [ ]:
# create new columns for Lat/Long/Elevation for the red and white white data
#There are three clusters of wineries. 
#The wineries in California, Washington, and Oregon are within a couple dozen miles of one another in each state.
#So I just used three stations: One for California, one for Washington, one for Oregon


#-------------------------------------------------------Station IDs---------------------------------------------------#
#California_GHCND:
Nappa_Valley_g='GHCND:USC00046074'
# Washington_GHCND:
Walla_Walla_Valley_g='GHCND:GHCND:USC00457015'
# Oregon_GHCND:
Willamette_Valley_g='GHCND:USW00024232'
#--------------------------------------------------------Latitude-----------------------------------------------------#

#Latitude
#California_LAT:
Cali_lat='38.2777'
# Oregon_lat:
Oregon_lat='44.905'
# Washington_Lat:
Washington_lat='46.3119'


#Create a new column in red_USA_df that has latitude for california, washingtion, oregon
latitude=[]
for lat in red_USA_df['regions']:
#California
    if lat =='California':
        latitude.append(Cali_lat)
#Oregon
    elif lat == 'Oregon':
        latitude.append(Oregon_lat)
#Washington
    else:
        latitude.append(Washington_lat)
red_USA_df['lat']=latitude

#Create a new column in white_USA_df that has latitude for california, washingtion, oregon
latitude=[]
for lat in white_USA_df['regions']:
#California
    if lat =='California':
        latitude.append(Cali_lat)
#Oregon
    elif lat == 'Oregon':
        latitude.append(Oregon_lat)
#Washington
    else:
        latitude.append(Washington_lat)
white_USA_df['lat']=latitude

#------------------------------------------------------Longitude-------------------------------------------------------------#

#Longitude
#California_Long:
Cali_long='-122.2647'
# Oregon_Long:
Oregon_long='-123.0011'
# Washington_Long:
Washington_long= '-119.2633'

#Create a new column in red_USA_df that has longitude for california, washingtion, oregon
longitude=[]
for long in red_USA_df['regions']:
#California
    if long =='California':
        longitude.append(Cali_long)
#Oregon
    elif long == 'Oregon':
        longitude.append(Oregon_long)
#Washington
    else:
        longitude.append(Washington_long)
red_USA_df['long']=longitude 

#Create a new column in white_USA_df that has longitude for california, washingtion, oregon
longitude=[]
for long in white_USA_df['regions']:
#California
    if long =='California':
        longitude.append(Cali_long)
#Oregon
    elif long == 'Oregon':
        longitude.append(Oregon_long)
#Washington
    else:
        longitude.append(Washington_long)
white_USA_df['long']=longitude 

#-----------------------------------------------------------------Elevation------------------------------------------------#

#Elevation (feet)
#California_Elevation:
Cali_ele='35.1'
# Oregon_Elevation:
Oregon_ele='205.1'
# Washington_Elevation:
Washington_ele= '631.9'


#Create a new column in red_USA_df that has elevation for california, washingtion, oregon
elevation=[]
for ele in red_USA_df['regions']:
#California
    if ele =='California':
        elevation.append(Cali_ele)
#Oregon
    elif ele == 'Oregon':
        elevation.append(Oregon_ele)
#Washington
    else:
        elevation.append(Washington_ele)
red_USA_df['elevation']=elevation

#Create a new column in white_USA_df that has elevation for california, washingtion, oregon
elevation=[]
for ele in white_USA_df['regions']:
#California
    if ele =='California':
        elevation.append(Cali_ele)
#Oregon
    elif ele == 'Oregon':
        elevation.append(Oregon_ele)
#Washington
    else:
        elevation.append(Washington_ele)
white_USA_df['elevation']=elevation


In [21]:
white_USA_df.head(2)

,wine,wine_id,appellation,color,wine_type,regions,country,classification,vintage,is_primeurs,score,confidence_index,journalist_count
0,"Ridge Vineyards, Monte Bello Chardonnay, White...",120785,Santa Cruz Mountains,White,dry,California,Usa,None,2016,False,94.39,B,3
1,"Stags' Leap Winery, Chardonnay, White, Napa Va...",129823,Napa Valley,White,dry,California,Usa,None,2016,False,89.83,A+,3


In [41]:
bigdf.reset_index(inplace=True)
bigdf['day'] = bigdf['date'].dt.day
bigdf['month'] = bigdf['date'].dt.month
bigdf['year'] = bigdf['date'].dt.year

list_months=[]
for x in bigdf['month']:
    list_months.append(calendar.month_name[x])
bigdf['months']=list_months
bigdf.drop(['date', 'day','month'], axis=1, inplace=True)
bigdf.reindex(['year','months','avgMaxTempCA','avgMinTempCA', 'avgMaxTempWash', 'avgMinTempWash'],axis=1)

,year,months,avgMaxTempCA,avgMinTempCA,avgMaxTempWash,avgMinTempWash
0,2010,January,49.645161,40.451613,44.806452,33.741935
1,2010,February,54.142857,39.000000,50.821429,33.785714
2,2010,March,56.838710,39.645161,60.741935,35.709677
3,2010,April,59.133333,42.766667,65.566667,42.133333
4,2010,May,63.225806,46.838710,69.645161,47.322581
5,2010,June,68.766667,52.100000,78.700000,55.600000
6,2010,July,79.032258,55.935484,90.225806,61.806452
7,2010,August,79.516129,56.774194,88.612903,61.806452
8,2010,September,73.000000,55.866667,77.433333,55.933333
9,2010,October,63.451613,46.709677,65.806452,46.064516


In [35]:
# loc is label based: 1, 2, 100--- you get rows that have a 1 or 2 or 100
# iloc is int based: 1, 2, 100---- you get the 1st, 2nd, 100th index
x=bigdf.loc[bigdf['months']=='January'].loc[bigdf['year']==2010]
x['avgMaxTempCA']

0    49.645161
Name: avgMaxTempCA, dtype: float64

In [ ]:
calender_years=[2013, 2014,2015, 2016, 2017]
calender_months=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']


In [ ]:
for i in 

In [ ]:

white_USA_df[month]=bigdf['avgMaxTempCA']['month'][month]

# Add Temp/Precipitation

In [ ]:
# Split the date into columns for days, months, years and then drop days columns for each of the three states
#-------------------------------------------------------------------------------California----------------------------
bigdf['day'] = bigdf['date'].dt.day
bigdf['month'] = bigdf['date'].dt.month
bigdf['year'] = bigdf['date'].dt.year

list_months=[]
for x in bigdf['month']:
    list_months.append(calendar.month_name[x])
bigdf['months']=list_months
bigdf.drop(['date', 'day','month'], axis=1, inplace=True)
bigdf
#----------------------------------------------------------------------------------Oregon-------------------------------
# df_temp_O.reset_index(inplace= True )
# df_temp_O['day'] = df_temp_O['date'].dt.day
# df_temp_O['month'] = df_temp_O['date'].dt.month
# df_temp_O['year'] = df_temp_O['date'].dt.year

# list_months=[]
# for x in df_temp_O['month']:
#     list_months.append(calendar.month_name[x])
# df_temp_O['months']=list_months
# df_temp_O.drop(['date', 'day','month'], axis=1, inplace=True)
# #-----------------------------------------------------------------------------------Washington---------------------
# df_temp_W.reset_index(inplace= True )
# df_temp_W['day'] = df_temp_W['date'].dt.day
# df_temp_W['month'] = df_temp_W['date'].dt.month
# df_temp_W['year'] = df_temp_W['date'].dt.year

# list_months=[]
# for x in df_temp_W['month']:
#     list_months.append(calendar.month_name[x])
# df_temp_W['months']=list_months
# df_temp_W.drop(['date', 'day','month'], axis=1, inplace=True)

In [ ]:
df_temp.head(2)

In [ ]:
red_USA_df

In [ ]:
calender_years=[2013, 2014,2015, 2016, 2017]
calender_months=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

#### California

In [ ]:
# temp_2013=[]
# temp_2014=[]
# temp_2015=[]
# temp_2016=[]
# temp_2017=[]
# temp_2018=[]
# garbage=[]
# prcp_2013=[]
# prcp_2014=[]
# prcp_2015=[]
# prcp_2016=[]
# prcp_2017=[]
# prcp_2018=[]

# for temp, prcp, year, month in zip(df_temp['avgTemp'], df_temp['avgPrcp'], df_temp['year'], df_temp['months']):
#     if year==2013:
#         temp_2013.append(temp)    
#     elif year==2014:
#         temp_2014.append(temp)
#     elif year==2015:
#         temp_2015.append(temp)
#     elif year==2016:
#         temp_2016.append(temp)
#     elif year==2017:
#         temp_2017.append(temp)
#     else:
#         garbage.append(temp)

# for temp, prcp, year, month in zip(df_temp['avgTemp'], df_temp['avgPrcp'], df_temp['year'], df_temp['months']):
#     if year==2013:
#         prcp_2013.append(temp)    
#     elif year==2014:
#         prcp_2014.append(temp)
#     elif year==2015:
#         prcp_2015.append(temp)
#     elif year==2016:
#         prcp_2016.append(temp)
#     elif year==2017:
#         prcp_2017.append(temp)
#     else:
#         garbage.append(temp)
# temp_2013

In [ ]:
# march_2016=[]
# april_2016=[]
# may_2016=[]
# june_2016=[]
# july_2016=[]
# august_2016=[]
# september_2016=[]


# for region,vint,year  in zip(white_USA_df['regions'],white_USA_df['vintage'],temp_2016):
#     if region=='California' and vint==2016:
#         march_2016.append(temp_2016[3])
#     elif region=='California' and vint==2016:
#         april_2016.append(temp_2016[4])
#     elif region=='California' and vint==2016:
#         may_2016.append(temp_2016[5])        
#     elif region=='California' and vint==2016:
#         june_2016.append(temp_2016[6])        
#     elif region=='California' and vint==2016:
#         july_2016.append(temp_2016[7])        
#     elif region=='California' and vint==2016:
#         august_2016.append(temp_2016[8])        
#     elif region=='California' and vint==2016:
#         september_2016.append(temp_2016[9])   
#     else:
#         garbage.append(0)
# march_2016        

In [ ]:
# list_temp_march=[]
# all_months=[]

# for region,vint,year, temp, month, cyears, cmonths  in zip(white_USA_df['regions'],white_USA_df['vintage'],df_temp['year'], df_temp['avgTemp'], df_temp['months'], calender_years, calender_months):
#     if region=='California' and vint==2016:
#             list_temp_march.append(temp)
#         else:
#             list_temp_march.append(0)
#     else:list_2016=[]
# march_2016=[]
# april_2016=[]
# may_2016=[]
# june_2016=[]
# july_2016=[]
# august_2016=[]
# september_2016=[]
# for region,vint,years,temps,month in zip(white_USA_df['regions'],white_USA_df['vintage'],df_temp['year'], df_temp['avgTemp'], df_temp['months']):
#     if vint==2016 and region=='Calfornia' and month=='March':
#         march_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='April':
#         april_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='May':
#         may_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='June':
#         june_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='July':
#         july_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='August':
#         august_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='September':
#         september_2016.append(temps) 
#         list_temp_march.append(1)
# list_temp_march
        

In [ ]:
# list_2016=[]
# march_2016=[]
# april_2016=[]
# may_2016=[]
# june_2016=[]
# july_2016=[]
# august_2016=[]
# september_2016=[]
# for region,vint,years,temps,month in zip(white_USA_df['regions'],white_USA_df['vintage'],df_temp['year'], df_temp['avgTemp'], df_temp['months']):
#     if vint==2016 and region=='Calfornia' and month=='March':
#         march_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='April':
#         april_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='May':
#         may_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='June':
#         june_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='July':
#         july_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='August':
#         august_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='September':
#         september_2016.append(temps) 

In [ ]:
# white_USA_df.head(2)

In [ ]:
# # Trying to add columns for weather data for each month

# list_temp_march=[]
# # def monthData (year, month, region, vintage):
# for u,v,x,y,z in zip(white_USA_df['regions'],white_USA_df['vintage'],df_temp['year'], df_temp['avgTemp'], df_temp['months']):
#     if v==2016 and v==2016 and u=='California'and z=='March':
#         list_temp_march.append(y)
#     #elif y==z and y==2016 and x=='Oregon'and v=='April':
#         #list_temp_march.append(u)
#     else:
#         list_temp_march.append(0)
            
# # white_USA_df['march']= list_temp_march         
# # white_USA_df['march']=white_USA_df['march'].apply(monthData)
# # list_temp_march    

In [ ]:
# # Trying to add columns for weather data for each month
# list_temp_march=[]
# march_2016=[]
# april_2016=[]





# # def monthData (year, month, region, vintage):
# for u,v,x,y,z in zip(white_USA_df['regions'],white_USA_df['vintage'],df_temp['year'], df_temp['avgTemp'], df_temp['months']):
#     if v==2016 and v==2016 and u=='California':
#         list_temp_march.append(y)
#     #elif y==z and y==2016 and x=='Oregon'and v=='April':
#         #list_temp_march.append(u)
#     else:
#         list_temp_march.append(0)
            
# # white_USA_df['march']= list_temp_march         
# # white_USA_df['march']=white_USA_df['march'].apply(monthData)
# list_temp_march

In [ ]:
sorted_red_USA_df=red_USA_df.sort_values(by=['regions'], ascending=False)
sorted_red_USA_df=sorted_red_USA_df[sorted_red_USA_df['regions']=='Oregon']
sorted_red_USA_df=sorted_red_USA_df.sort_values(by=['vintage'], ascending=True)
sorted_red_USA_df.head(50)

Imported dependencies<p>
Used the Global Wine Score API to import data for red and white wine in the USA<p>
Imported weather data for California, Oregon, Washington<p>
Added new columns Latitude, Longitude, and Elevation to red and white wine dataframes<p>
Finally I am trying to add new columns for monthly weather data to the red and white wine dataframes<p>

In [ ]:
# # .apply() method
# white_USA_df['march']=[]

#axis=1 for columns
#default--->result_type=None means that final return type is inferred (default)
#raw is false means that it will be a series, True is it is an array

In [ ]:
### initialize lists to store data
dates_prcp_W = []
prcp_W = []

#for each year from 2015-2019 ...
for year in range(1992, 2017):
    year = str(year)
#     print('working on year '+year)
    
    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USC00046074&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are Precipitation temperature readings
    avg_prcp = [item for item in d['results'] if item['datatype']=='PRCP']
    #get the date field from all average Precipitation readings
    dates_prcp_W += [item['date'] for item in avg_prcp]
    
    
    #get the actual average Precipitation from all average Precipitation readings
    prcp_W += [item['value'] for item in avg_prcp]

#initialize dataframe
df_prcp_W = pd.DataFrame()

#populate date and average Precipitation fields (cast string date to datetime
df_prcp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_W]
#convert to mm to inches
df_prcp_W['avgPrcp'] = [float(v)*0.0393701 for v in prcp_W]
df_prcp_W = df_prcp_W.set_index('date').resample('M').mean()
df_prcp_W